In [116]:
from sage.all import *

# Параметры кода
q = 8  # Порядок поля
k_0
#k_0 = 1   Степень полинома
n = q + 1  # Локальная длина кода
N = (q + 1) * (q**2 + q + 1)  # Глобальная длина кода
F = GF(q)  # Конечное поле
R.<x, y, z> = PolynomialRing(F)  # Для использования x, y, z для однородных полиномов
k = k_0 + 1

# Генерация проективной плоскости
def generate_projective_plane(F):
    points = []
    lines = []
    for x in F:
        for y in F:
            for z in F:
                if (x, y, z) != (0, 0, 0):  # Исключаем точку (0, 0, 0)
                    is_unique = True
                    for a in F:
                        if a != 0:
                            # Масштабируем точку на элементы поля F
                            scaled_point = (a * x, a * y, a * z)
                            if scaled_point in points:
                                is_unique = False
                                break
                    if is_unique:
                        points.append((x, y, z))  # Добавляем точку в список
                        lines.append((x, y, z))   # Прямую тоже
    return points, lines

# Генерация точек и прямых
points, lines = generate_projective_plane(F)

# Индексация рёбер для быстрого доступа
def incidence_matrix(points, lines, F):
    incidence = Matrix(F, len(points), len(lines))
    edges = []
    edge_numbers = {}
    point_numbers = {point: i for i, point in enumerate(points)}  # Словарь для точек
    line_numbers = {line: j for j, line in enumerate(lines)}  # Словарь для прямых
    
    for i, point in enumerate(points):
        for j, line in enumerate(lines):
            if sum(F(point[k]) * F(line[k]) for k in range(3)) == 0:  # Проверка инцидентности
                incidence[i, j] = 1
                edge_numbers[(i, j)] = len(edges)
                edges.append((i, j))  # Добавление ребра в список
    
    return incidence, edges, edge_numbers, point_numbers, line_numbers

# Создаем матрицу инцидентности и словари
I, edges, edge_numbers, point_numbers, line_numbers = incidence_matrix(points, lines, F)

# Функция для безопасного возведения в степень
def safe_power(base, exp):
    if base == 0 and exp == 0:
        return F(1)  # Определяем 0^0 как 1
    else:
        return base**exp

# Функция для генерации проверочной матрицы для прямой с учётом инцидентности
def parity_check_matrix_for_line(N, n, k, line, points, incidence_matrix, edge_numbers, line_numbers):
    matrix_rows = []
    for i in range(0, n - k):  # Генерация строк для полиномов
        for j in range(0, n - k):
            if i + j >= n - k:
                continue
            row = [0] * N
            # Для каждой точки, инцидентной данной прямой
            for s, point in enumerate(points):
                if incidence_matrix[s, line_numbers[line]] == 1:  # Проверяем инцидентность
                    val = safe_power(F(point[0]), i) * safe_power(F(point[1]), j) * safe_power(F(point[2]), n - k - 1 - i - j)
                    row[edge_numbers[(s, line_numbers[line])]] = val  # Используем словарь для быстрого доступа
            matrix_rows.append(row)
    # Удалим линейно зависимые строки
    rk = Matrix(matrix_rows).rank()
    matrix_rows = Matrix(matrix_rows).echelon_form().rows()[:rk]
    return Matrix(F, matrix_rows)

# Функция для генерации проверочной матрицы для точки с учётом инцидентности
def parity_check_matrix_for_point(N, n, k, point, lines, incidence_matrix, edge_numbers, point_numbers, line_numbers):
    matrix_rows = []
    for i in range(0, n - k):  # Генерация строк для полиномов
        for j in range(0, n - k):
            if i + j >= n - k:
                continue
            row = [0] * N
            # Для каждой прямой, инцидентной данной точке
            for s, line in enumerate(lines):
                if incidence_matrix[point_numbers[point], s] == 1:  # Проверяем инцидентность
                    val = safe_power(F(line[0]), i) * safe_power(F(line[1]), j) * safe_power(F(line[2]), n - k - 1 - i - j)  # Генерация полинома
                    row[edge_numbers[(point_numbers[point], line_numbers[line])]] = val  # Используем словарь для быстрого доступа
            matrix_rows.append(row)
    # Удалим линейно зависимые строки
    rk = Matrix(matrix_rows).rank()
    matrix_rows = Matrix(matrix_rows).echelon_form().rows()[:rk]
    return Matrix(F, matrix_rows)

# Функция для генерации матрицы Таннера
def tanner_matrix(incidence_matrix, n, k_0, points, lines, edge_numbers, point_numbers, line_numbers):
    tanner_matrix = []  # Используем список списков для матрицы
    N = (q**2 + q + 1) * (q + 1)
    k = k_0 + 1

    # Заполнение верхней части матрицы для точек
    for i in range(q**2 + q + 1):  # Это количество точек
        point_matrix = parity_check_matrix_for_point(N, n, k, points[i], lines, incidence_matrix, edge_numbers, point_numbers, line_numbers)
        tanner_matrix += point_matrix

    # Заполнение нижней части матрицы для прямых
    for i in range(q**2 + q + 1):  # Это количество прямых
        line_matrix = parity_check_matrix_for_line(N, n, k, lines[i], points, incidence_matrix, edge_numbers, line_numbers)
        tanner_matrix += line_matrix

    return Matrix(F, tanner_matrix)




In [117]:
import csv
import os


def center_text(text, width):
    return str(text).center(width)


def test_for_k_0_range(q):
    results = []  
    for k_0 in range(0, q):
       
        T = tanner_matrix(I, n, k_0, points, lines, edge_numbers, point_numbers, line_numbers)
        rk = T.rank()
        tanner_matrix_dim_rows = T.nrows()  
        tanner_matrix_dim_cols = T.ncols()  

        """
        print(f"q: {q}")
        print(f"k_0: {k_0}")
        print(f"Ранг матрицы Таннера: {rk}")
        print(f"Размерность по строкам: {tanner_matrix_dim_rows}")
        print(f"Размерность по столбцам: {tanner_matrix_dim_cols}")
        """
      
        l = N - rk
        
        """
        print(f"Размерность кода  l = N - rk: {l}")
        print("-------------------------")
        print()
        """
       
        results.append([q, k_0, rk, l, f"{tanner_matrix_dim_rows} x {tanner_matrix_dim_cols}"])

    
    file_name = "tanner_matrix_results.csv"
    
   
    with open(file_name, mode='a', newline='') as file:
        writer = csv.writer(file)

       
        writer.writerow(["q", "k_0", "Ранг", "Размерность кода", "Размерность матрицы Таннера (строки x столбцы)"])

        # Вычислим максимальную ширину каждого столбца, чтобы данные выравнивались
        column_widths = [max(len(str(cell)) for cell in col) for col in zip(*results)]
        
        # Применяем выравнивание по центру для всех данных, используя вычисленные ширины
        centered_results = [
            [center_text(cell, column_widths[i]) for i, cell in enumerate(row)]
            for row in results
        ]
        
        
        writer.writerows(centered_results)

    print(f"Результаты сохранены в '{file_name}'.")
    

test_for_k_0_range(q)


Результаты сохранены в 'tanner_matrix_results.csv'.


In [118]:
import csv
with open('tanner_matrix_results.csv', newline='') as file:
        reader = csv.reader(file)
        
       
        headers = next(reader)
        
      
        print("{:<5} {:<5} {:<5} {:<30} {:<30}".format(*headers))  # Применяем выравнивание для заголовков
        row_count = 0
        
        for row in reader:
            print("{:<5} {:<5} {:<5} {:<30} {:<31}".format(*row))  # Применяем выравнивание для данных
            row_count += 1 
            #print(row_count)

q     k_0   Ранг  Размерность кода               Размерность матрицы Таннера (строки x столбцы)
2     0     20    1                              28 x 21                        
2     1     13    8                              14 x 21                        
q     k_0   Ранг  Размерность кода               Размерность матрицы Таннера (строки x столбцы)
3     0     51    1                              78 x 52                        
3     1     44    8                              52 x 52                        
3     2     25    27                             26 x 52                        
q     k_0   Ранг  Размерность кода               Размерность матрицы Таннера (строки x столбцы)
4     0     104   1                              168 x 105                      
4     1      97   8                              126 x 105                      
4     2      76   29                              84 x 105                      
4     3      41   64                              42 x 105      

если уже смотрели такое q и нужно удалить какие -то строки, чтобы не было никаких повторов в файле

In [ ]:
import csv

def remove_specific_rows_from_csv(file_name, rows_to_remove):
    with open(filename, newline =''):
        reader = csv.reader(file)
        all_rows = list(reader)
    
    filtered_rows = [row for i in enumerate(all_rows) if i not in rows_to_remove]
        
    with open('tanner_matrix_results.csv', mode='w', newline=''):
        writer = csv.writer(file)
        writer.writerows(filtered_rows)
        
    print(f"Удалены строки {rows_to_remove} из '{file_name}'.")
#remove_specific_rows_from_csv("tanner_matrix_results.csv", rows_to_remove=[1, 3])

на всякий добавлю, если вывод не устраивает глобально в файле

In [96]:
with open('tanner_matrix_results.csv', mode='w', newline=''):
    pass
    